## Importing Libraries

In [1]:
# Data processing
import numpy as np 
import pandas as pd 

# Data visualization
import matplotlib.pyplot as plt 
import chart_studio
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

In [2]:
# Makes Jupyter Notebook cells wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Print the full contents of each dataframe cell
pd.set_option('display.max_colwidth', None)

## Importing Data

In [3]:
WTTS_df = pd.read_excel('WTTS_Info.xlsx')

In [4]:
most_recent_reviewed_episode = 193

ratings_df = WTTS_df.iloc[:most_recent_reviewed_episode, :6]

In [5]:
ratings_df

,Episode_Total,Season_Num,Episode_Num,Title,Original_Chevrons,Second_Chance
0,1,1,1,Children of the Gods,"4,4","6,4"
1,2,1,2,The Enemy Within,"2,5",NaN
2,3,1,3,Emancipation,"1,0.5","0,0"
3,4,1,4,The Broca Divide,"6,5",NaN
4,5,1,5,The First Commandment,"3,2.5",NaN
...,...,...,...,...,...,...
188,189,9,16,Off the Grid,"7,7",NaN
189,190,9,17,The Scourge,"2,2.5",NaN
190,191,9,18,Arthur's Mantle,"5,5",NaN
191,192,9,19,Crusade,"5,5.5",NaN


## Data Transformation

In [6]:
# Convert all values within a column to tuples, if the value is not null
def transform_column_values_to_tuples(df: pd.DataFrame, column_name: str):
    df[column_name] = df[column_name].apply(lambda x: tuple(map(float, x.split(','))) if pd.notnull(x) else float('NaN'))
    
    
# Convert 'Original_Chevrons' column to tuples
transform_column_values_to_tuples(ratings_df, 'Original_Chevrons')
# Convert 'Second_Chance' column to tuples
transform_column_values_to_tuples(ratings_df, 'Second_Chance')

## Creating New Features

In [7]:
# Create a list with the current ratings of each episode, accounting for second chance ratings
updated_ratings = np.where(pd.isna(ratings_df['Second_Chance']), # If Second_Chance is NaN
                                   ratings_df['Original_Chevrons'], # select Chevron_Ratings
                                   ratings_df['Second_Chance']) # otherwise select Second_Chance


# Extract the first host's ratings into a new column
ratings_df['Brent_Chevrons'] = [rating[0] for rating in updated_ratings]

# Extract the second host's ratings into a new column
ratings_df['Zach_Chevrons'] = [rating[1] for rating in updated_ratings]


# Remove column Second_Chance, as it will not be used again
ratings_df.drop('Second_Chance', axis = 1, inplace = True)

In [8]:
# Create a list with the difference between the host's scores
ratings_df['Rating_Differential'] = abs(ratings_df['Brent_Chevrons'] - ratings_df['Zach_Chevrons'])

In [9]:
ratings_df

,Episode_Total,Season_Num,Episode_Num,Title,Original_Chevrons,Brent_Chevrons,Zach_Chevrons,Rating_Differential
0,1,1,1,Children of the Gods,"(4.0, 4.0)",6.0,4.0,2.0
1,2,1,2,The Enemy Within,"(2.0, 5.0)",2.0,5.0,3.0
2,3,1,3,Emancipation,"(1.0, 0.5)",0.0,0.0,0.0
3,4,1,4,The Broca Divide,"(6.0, 5.0)",6.0,5.0,1.0
4,5,1,5,The First Commandment,"(3.0, 2.5)",3.0,2.5,0.5
...,...,...,...,...,...,...,...,...
188,189,9,16,Off the Grid,"(7.0, 7.0)",7.0,7.0,0.0
189,190,9,17,The Scourge,"(2.0, 2.5)",2.0,2.5,0.5
190,191,9,18,Arthur's Mantle,"(5.0, 5.0)",5.0,5.0,0.0
191,192,9,19,Crusade,"(5.0, 5.5)",5.0,5.5,0.5


### Brent's Least Favorite Episodes

In [10]:
ratings_df.sort_values(by = ['Brent_Chevrons', 'Season_Num', 'Episode_Num']).head(14).reset_index()[['Season_Num', 'Episode_Num', 'Title', 'Brent_Chevrons']]

,Season_Num,Episode_Num,Title,Brent_Chevrons
0,1,3,Emancipation,0.0
1,1,2,The Enemy Within,2.0
2,1,8,Brief Candle,2.0
3,1,20,Politics,2.0
4,2,10,Bane,2.0
5,2,13,Spirits,2.0
6,2,19,One False Step,2.0
7,3,2,Seth,2.0
8,3,13,The Devil You Know,2.0
9,5,20,The Sentinel,2.0


### Zach's Least Favorite Episodes

In [11]:
ratings_df.sort_values(by = ['Zach_Chevrons', 'Season_Num', 'Episode_Num']).head(12).reset_index()[['Season_Num', 'Episode_Num', 'Title', 'Zach_Chevrons']]

,Season_Num,Episode_Num,Title,Zach_Chevrons
0,1,3,Emancipation,0.0
1,1,15,Cor-ai,2.0
2,2,10,Bane,2.0
3,2,13,Spirits,2.0
4,2,19,One False Step,2.0
5,3,2,Seth,2.0
6,8,5,Icon,2.0
7,1,5,The First Commandment,2.5
8,1,20,Politics,2.5
9,2,22,Out of Mind,2.5


### Brent's Favorite Episodes

In [12]:
ratings_df.sort_values(by = ['Brent_Chevrons', 'Season_Num', 'Episode_Num'], ascending = [False, True, True]).head(7).reset_index()[['Season_Num', 'Episode_Num', 'Title', 'Brent_Chevrons']]

,Season_Num,Episode_Num,Title,Brent_Chevrons
0,2,15,The Fifth Race,8.0
1,5,21,Meridian,8.0
2,6,12,Unnatural Selection,8.0
3,6,22,Full Circle,8.0
4,8,11,Gemini,8.0
5,8,19,Moebius: Part 1,8.0
6,8,20,Moebius: Part 2,8.0


### Zach's Favorite Episodes

In [13]:
ratings_df.sort_values(by = ['Zach_Chevrons', 'Season_Num', 'Episode_Num'], ascending = [False, True, True]).head(12).reset_index()[['Season_Num', 'Episode_Num', 'Title', 'Zach_Chevrons']]

,Season_Num,Episode_Num,Title,Zach_Chevrons
0,2,15,The Fifth Race,8.0
1,5,12,Wormhole X-Treme!,8.0
2,5,21,Meridian,8.0
3,6,19,The Changeling,8.0
4,6,22,Full Circle,8.0
5,7,17,Heroes: Part 1,8.0
6,7,18,Heroes: Part 2,8.0
7,7,21,Lost City: Part 1,8.0
8,7,22,Lost City: Part 2,8.0
9,8,18,Threads,8.0


In [14]:
# Create figure
fig = go.Figure()

# Create list of hover texts using custom hover text template by iterating over every data row
hover_texts = [f'{row.Title}<br>Rating: {row.Brent_Chevrons}' for row in ratings_df.itertuples(index=False)]

fig.add_trace(go.Scatter(x = ratings_df.index,               # x-axis data (Episode number)
                         y = ratings_df['Brent_Chevrons'],   # y-axis data (Brent's ratings)
                         mode = 'markers',                   # Trace type
                         name = 'Brent',                     # Custom legend text
                         marker = dict(color = '#0096FF'),   # Custom marker color
                         text = hover_texts,                 # Custom hover text
                         hoverinfo = 'text'))                # Only display custom text

# Create list of hover texts using custom hover text template by iterating over every data row
hover_texts = [f'{row.Title}<br>Rating: {row.Zach_Chevrons}' for row in ratings_df.itertuples(index=False)]

fig.add_trace(go.Scatter(x = ratings_df.index,               # x-axis data (Episode number)
                         y = ratings_df['Zach_Chevrons'],    # y-axis data (Zach's ratings)
                         mode = 'markers',                   # Trace type
                         name = 'Zach',                      # Custom legend text
                         marker = dict(color = '#FFA500'),   # Custom marker color
                         text = hover_texts,                 # Custom hover text
                         hoverinfo = 'text'))                # Only display custom text

#Further style the figure
fig.update_layout(
    title = 'Chevron Ratings',        # Set plot title
    xaxis_title = 'Episode Number',   # Set x-axis title
    yaxis_title = 'Rating',           # Set y-axis title
    yaxis = dict(range = [0, 9]),     # Set y-axis range
    paper_bgcolor = '#011222',        # Set background color
    font_color = '#ffffff')           # Set font color

# Display figure
fig.show()

# Create plot in Chart Studio for later embedding
py.plot(fig, filename = 'Chevron Ratings', auto_open = False)

'https://plotly.com/~Cbsiros/8/'

### How do Brent and Zach's scores compare over time? 

In [15]:
# Create figure
fig = go.Figure()

fig.add_trace(go.Scatter(x = ratings_df.index,                                           # x-axis data (Episode number)
                         y = ratings_df['Brent_Chevrons'].expanding().mean().round(3),   # y-axis data (Expanding average rating, rounded to three decimals)
                         mode = 'lines',                                                 # Trace type
                         name = 'Brent',                                                 # Custom legend text
                         line = dict(color = '#0096FF', width = 2)))                     # Custom line color and width

fig.add_trace(go.Scatter(x = ratings_df.index,                                          # x-axis data (Episode number)
                         y = ratings_df['Zach_Chevrons'].expanding().mean().round(3),   # y-axis data (Expanding average rating, rounded to three decimals)
                         mode = 'lines',                                                # Trace type
                         name = 'Zach',                                                 # Custom legend text
                         line = dict(color = '#FFA500', width = 2)))                    # Custom line color and width

#Further style the figure
fig.update_layout(
    title = 'Average Chevron Ratings Over Time',   # Set plot title
    xaxis_title = 'Episode Number',                # Set x-axis title
    yaxis_title = 'Rating',                        # Set y-axis title
    yaxis = dict(range = [0, 8]),                  # Set y-axis range
    paper_bgcolor = '#011222',                     # Set background color
    font_color = '#ffffff')                        # Set font color

# Display figure
fig.show()

# Create plot in Chart Studio for later embedding
py.plot(fig, filename = 'Average Chevron Ratings Over Time', auto_open = False)

'https://plotly.com/~Cbsiros/11/'

# The Differences in the Hosts' Scores

### What episodes did Brent and Zach disagree on the most? 

In [16]:
ratings_df.sort_values('Rating_Differential', ascending = False).head(11).reset_index()[['Season_Num', 'Episode_Num', 'Title', 'Brent_Chevrons', 'Zach_Chevrons', 'Rating_Differential']]

,Season_Num,Episode_Num,Title,Brent_Chevrons,Zach_Chevrons,Rating_Differential
0,7,6,Lifeboat,7.0,3.0,4.0
1,4,15,Chain Reaction,3.0,7.0,4.0
2,1,2,The Enemy Within,2.0,5.0,3.0
3,7,13,Grace,7.0,4.0,3.0
4,5,13,Proving Ground,7.0,4.0,3.0
5,7,15,Chimera,6.0,3.0,3.0
6,4,1,Small Victories,4.0,7.0,3.0
7,4,20,Entity,6.0,3.5,2.5
8,5,5,Red Sky,7.0,4.5,2.5
9,6,16,Metamorphosis,5.0,2.5,2.5


In [17]:
# Create a list of lists that holds the rating differentials for each episode grouped by seasons, to be used in upcoming heatmap
rating_differentials = []

# Create a list of lists that holds the hover text for each episode grouped by seasons, to be used in upcoming heatmap
hover_texts = []

# Loop through every season
for i in ratings_df['Season_Num'].unique():
    
    # Collect every episode title in this season
    episode_titles = ratings_df.loc[ratings_df['Season_Num'] == i, 'Title'].values
    
    # Collect every rating differential in this season
    episode_rating_differences = ratings_df.loc[ratings_df['Season_Num'] == i, 'Rating_Differential'].values
    
    # Add rating differentials to global list for use later
    rating_differentials.append(episode_rating_differences)
    
    # Create a list to hold hover_texts for each episode in this season
    season_hover_texts = []
    
    # Loop through every episode in this season
    for j in range(len(episode_titles)):
        
        # If/else statement for grammer purposes (1 Chevron vs 2 Chevrons), which appends the hover text for this episode to season_hover_texts
        if(episode_rating_differences[j]) == 1:
            season_hover_texts.append(f'Season {i}, Episode {j + 1}<br>{episode_titles[j]}<br>Difference: {episode_rating_differences[j]} Chevron')
        else:
            season_hover_texts.append(f'Season {i}, Episode {j + 1}<br>{episode_titles[j]}<br>Difference: {episode_rating_differences[j]} Chevrons')
    
    # Finally, append each season_hover_text to hover_texts for use in heatmap
    hover_texts.append(season_hover_texts)

In [18]:
# Create heatmap figure
fig = go.Figure(data = go.Heatmap(z = rating_differentials,                                          # z-axis data (Difference in episode ratings)
                                  x = list(range(1, len(ratings_df['Episode_Num'].unique()) + 1)),   # x-axis data (Episode numbers)
                                  y = list(range(1, ratings_df['Season_Num'].max() + 1)),            # y-axis data (Corresponding season numbers)
                                  hoverongaps = False,                                              # Do not show info for missing values
                                  text = hover_texts,                                                # Custom hover text
                                  hoverinfo = 'text'))                                               # Only display custom text

#Further style the figure
fig.update_layout(
    title = 'Differences in the Host Ratings',   # Set plot title
    xaxis_title = 'Episode Number',              # Set x-axis title
    yaxis_title = 'Season Number',               # Set y-axis title
    paper_bgcolor = '#011222',                   # Set background color
    font_color = '#ffffff')                      # Set font color

# Display figure
fig.show()

# Create plot in Chart Studio for later embedding
py.plot(fig, filename = 'Differences in the Host Ratings', auto_open = False)

'https://plotly.com/~Cbsiros/13/'

In [19]:
# Create figure
fig = go.Figure()

fig.add_trace(go.Scatter(x = ratings_df.index,                                                # x-axis data (Episode numbers)
                         y = ratings_df['Rating_Differential'].expanding().mean().round(3),   # y-axis data (Expanding average difference between host scores, rounded to three decimal places)
                         mode = 'lines',                                                      # Trace type
                         line = dict(color = '#DA9100', width = 2)))                          # Custom line color and width

#Further style the figure
fig.update_layout(
    title = 'Average Difference Between the Hosts\' Scores Over Time',   # Set x-plot title
    xaxis_title = 'Episode Number',                                      # Set x-axis title
    yaxis_title = 'Average Score Difference',                            # Set y-axis title
    paper_bgcolor = '#011222',                                           # Set background color
    font_color = '#ffffff',                                              # Set font color
    showlegend = False)                                                  # Do not display legend


# Display figure
fig.show()

# Create plot in Chart Studio for later embedding
py.plot(fig, filename = 'Average Difference Between the Hosts\' Scores Over Time', auto_open = False)

'https://plotly.com/~Cbsiros/15/'

In [32]:
highest_reviewer_counts

Tie      81
Zach     58
Brent    54
Name: count, dtype: int64

### Distribution of Higher Episode Ratings Between Hosts

In [20]:
# Create labels for pie chart
labels = ['Tie','Zach','Brent']

# Create data for pie chart, based on which host rated episodes higher
highest_reviewer = ['Brent' if B > Z else 'Zach' if B < Z else 'Tie' for B, Z in zip(ratings_df['Brent_Chevrons'], ratings_df['Zach_Chevrons'])]

# Count number of occurances in previous data
highest_reviewer_counts = pd.Series(highest_reviewer).value_counts()

# Create figure
fig = go.Figure(data = go.Pie(labels = labels,                    # Pie chart labels
                              values = highest_reviewer_counts,   # Pie chart values
                              hole = 0.2))                        # Create hole in the middle of the chart

# Create list of colors for pie chart
colors = ['#90EE90', 'FFA500', '#0096FF']

fig.update_traces(hoverinfo = 'label',                                                         # Set info upon hover to slice label
                  textinfo = 'percent',                                                        # Set text within slice to percent
                  textfont_size = 20,                                                          # Set font size
                  marker = dict(colors = colors, line = dict(color = '#ffffff', width = 2)))   # Set colors of slices and lines


#Further style the figure
fig.update_layout(
    title = 'Distribution of Higher Episode Ratings Between Hosts',   # Set plot title
    xaxis_title = 'Episode Number',             # Set x-axis title
    yaxis_title = 'Average Score Difference',   # Set y-axis title
    paper_bgcolor = '#011222',                  # Set background color
    font_color = '#ffffff')                     # Set font color;

# Display figure
fig.show()

# Create plot in Chart Studio for later embedding
py.plot(fig, filename = 'Distribution of Higher Episode Ratings Between Hosts', auto_open = False)

'https://plotly.com/~Cbsiros/17/'

## Season Long Ratings

In [21]:
# Create a new dataframe of ratings grouped by the season number
seasons_df = ratings_df.groupby('Season_Num').mean(numeric_only=True)[['Brent_Chevrons', 'Zach_Chevrons']]

In [22]:
# Create data for bar chart x-axis
seasons = seasons_df.index

# Create figure
fig = go.Figure()

fig.add_trace(go.Bar(x = seasons,                        # x-axis data (Season numbers)
                     y = seasons_df['Brent_Chevrons'],   # y-axis data (Brent's average chevron rating of each season)
                     name = 'Brent',                     # Custom legend text
                     marker_color = '#0096FF'))          # Set color of bars

fig.add_trace(go.Bar(x = seasons,                       # x-axis data (Season numbers)
                     y = seasons_df['Zach_Chevrons'],   # y-axis data (Zach's average chevron rating of each season)
                     name = 'Zach',                     # Custom legend text
                     marker_color = '#FFA500'))         # Set color of bars

fig.update_layout(
    title_text = 'Average Chevron Rating by Season',   # Set plot title
    xaxis_title_text = 'Season',                       # Set x-axis title
    yaxis_title_text = 'Average Chevron Rating',       # Set y-axis title
    paper_bgcolor = '#011222',                         # Set background color
    font_color = '#ffffff',                            # Set font color
    bargap = 0.2,                                      # Set gap size between groups
    bargroupgap = 0.1)                                 # Set gap size within groups


# Display figure
fig.show()

# Create plot in Chart Studio for later embedding
py.plot(fig, filename = 'Average Chevron Rating by Season', auto_open = False)

'https://plotly.com/~Cbsiros/19/'

## Working with Listener Predictions

In [23]:
# Create dataframe with predictions
predictions_df = WTTS_df.iloc[:most_recent_reviewed_episode, 6:]

# Convert all predictions to tuples by iterating through each column
for column in predictions_df.columns:
    transform_column_values_to_tuples(predictions_df, column)

In [24]:
predictions_df

,David,Kevin,Kree Amisk,Sean,Paul,Dinner Beef,Dimitri,Rowan,Jen,LydiaAnn,...,Dustin,Luther,Cassidy,Letia,Fraiser,Kim,Mcjeston,Shawn,Arin,Mad Baron
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,"(4.0, 4.5)","(5.0, 5.0)","(5.0, 5.5)","(4.0, 4.0)","(5.0, 5.5)","(5.0, 5.0)","(5.0, 5.0)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,"(6.0, 6.0)","(4.0, 4.0)","(3.0, 4.0)","(7.0, 3.0)",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,"(3.0, 4.0)","(4.0, 4.0)","(3.0, 4.5)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,"(6.0, 6.5)","(3.0, 4.0)","(4.0, 5.0)","(4.0, 4.5)","(6.0, 5.5)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Count the number of correct predictions
def count_correct_predictions(targets: pd.Series, predictions: pd.Series) -> int:
    correct = 0
    
    for target, prediction in zip(targets, predictions):
        # Check to ensure value is not null and for correctness
        if not pd.isna(prediction) and prediction[0] == target[0] and prediction[1] == target[1]:
            correct += 1
            
    return correct

# Count the number of  predictions
def count_predictions(predictions: pd.Series) -> int:
            
    return predictions.count()

In [26]:
data = []

# Loop through every listener
for name in predictions_df.columns:
    num_correct_predictions = count_correct_predictions(ratings_df['Original_Chevrons'], predictions_df[name])
    num_predictions = count_predictions(predictions_df[name])
    percent_correct = round(num_correct_predictions/num_predictions * 100, 2)
    
    data.append((num_correct_predictions, num_predictions, percent_correct))
                    
                
# Create a dataframe from data list
prediction_stats_df = pd.DataFrame(data, columns = ['Correct', 'Total', 'Percent'])

# Set indexes to listener names
prediction_stats_df.index = list(predictions_df.columns)

# Manually adding a correct score to David for 5.16.Summit/5.17.Last Stand (On behalf of the hosts)
prediction_stats_df.loc['David', 'Correct'] += 1
prediction_stats_df.loc['David', 'Percent'] = round(prediction_stats_df.loc['David', 'Correct']/prediction_stats_df.loc['David', 'Total'] * 100, 2)

In [27]:
# Slice dataframe to include listeners with at least one correct prediction
prediction_stats_df = prediction_stats_df[prediction_stats_df['Correct'] > 0]

# Sort dataframe by most correct predictions
prediction_stats_df = prediction_stats_df.sort_values('Correct', ascending = False)


# Create figure
fig = go.Figure()

fig.add_trace(go.Bar(x = prediction_stats_df.index,        # x-axis data (Listener Names)
                     y = prediction_stats_df['Correct'],   # y-axis data (Number of correct predictions)
                     marker_color = '#0096FF'))            # Set color of bars

fig.update_layout(
    title_text = 'Correct Predictions By Listeners',          # Set plot title
    xaxis_title_text = 'Listener',                            # Set x-axis title
    yaxis_title_text = 'Number of Correct Predictions',       # Set y-axis title
    paper_bgcolor = '#011222',                                # Set background color
    font_color = '#ffffff',                                   # Set font color
    bargap = 0.2,                                             # Set gap size between groups
    bargroupgap = 0.1)                                        # Set gap size within groups


# Display figure
fig.show()

# Create plot in Chart Studio for later embedding
py.plot(fig, filename = 'Correct Predictions By Listeners', auto_open = False)

'https://plotly.com/~Cbsiros/25/'

In [31]:
# Create hover text for scatter plot
hover_texts = [f"Listener: {listener}<br>Percent Correct: {correct_percent}%<br>Total Predictions: {total}"
               for listener, correct_percent, total in zip(prediction_stats_df.index, prediction_stats_df['Percent'], prediction_stats_df['Total'])]

# Manually modify hover texts, to ensure overlapping entries are noted
hover_texts[-1] = "Listener: Mcjeston & Carrie<br>Percent Correct: 100%<br>Total Predictions: 1"


# Create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x = prediction_stats_df['Total'] ,        # x-axis data (Number of total guesses)
                         y = prediction_stats_df['Percent']  ,     # y-axis data (Percent of predictions that are correct)
                         mode = 'markers',                         # Trace type
                         marker = dict(color = '#0096FF'),         # Custom marker color
                         hovertext = hover_texts,                  # Custom hover text
                         hoverinfo = 'text'))                      # Only display custom text
                         


fig.update_layout(
    title_text = 'Total Predictions vs Correct Predictions',        # Set plot title
    xaxis_title_text = 'Number of Predictions',                     # Set x-axis title
    yaxis_title_text = 'Percent of Predictions that are Correct',   # Set y-axis title
    paper_bgcolor = '#011222',                                      # Set background color
    font_color = '#ffffff',                                         # Set font color
    bargap = 0.2,                                                   # Set gap size between groups
    bargroupgap = 0.1)                                              # Set gap size within groups


# Display figure
fig.show()

# Create plot in Chart Studio for later embedding
#py.plot(fig, filename = 'Percent Correct Predictions By Listeners', auto_open = False)

'https://plotly.com/~Cbsiros/30/'